In [10]:
# new low power ESP32 going here 
# top is the mqtt retrieve interface
# bottom is the low power deepsleep cycle saving to data

# We have mqttoperation.py which is imported when the pin12 switch is pulled
# Need a function in the operation for receiving timestamp and 
# renaming all the files with timestamp in the name
# so that we can stream back the correctly offset values 
# to backfill the values into grafana consistently

# then also begin discarding old files and ones that did not 
# pick up the time (eg reset_cause was not timeout, or pinwake)
# (but is a power reset so the clock is reset)

# The pins along the right hand side of the ESP32 board with wifi up are:
# 3.3V  TX  RX  GND  21  18  17  4  2
# left hand side second pin down is 35

In [259]:
%serialconnect

Connecting to --port=/dev/ttyUSB5 --baud=115200 
Ready.


In [35]:
import network
network.WLAN().active(0)

state: 5 -> 0 (0)
rm 0
del if0
mode : softAP(3e:71:bf:2c:aa:08)


In [336]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB5 --baud=115200 
Ready.


In [337]:
from machine import I2C, Pin
pinscl, pinsda = Pin(18), Pin(17)
#pinscl, pinsda = Pin(5), Pin(4)  # D1 D2
i2c = I2C(scl=pinscl, sda=pinsda, freq=100000)

print(i2c.scan(), hex(80), hex(104))
r = i2c.readfrom_mem(0x68, 0x00, 7)
print(r)
def rhex(h):  
    return (h>>4)*10 + (h&0x0f)
year, month, day = rhex(r[6])+2000, rhex(r[5]), rhex(r[4])
hour, minute, second = rhex(r[2]), rhex(r[1]), rhex(r[0])


[80, 104] 0x50 0x68
b'\xd8V\x15\x01\x12\x12 '


In [235]:
print(i2c.freq())

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
AttributeError: 'I2C' object has no attribute 'freq'


In [234]:
freq=100000

['__class__', 'init', 'readfrom', 'readfrom_into', 'readfrom_mem', 'readfrom_mem_into', 'readinto', 'scan', 'start', 'stop', 'write', 'writeto', 'writeto_mem']


In [230]:
r = i2c.readfrom_mem(0x68, 0x00, 7)
print(r)


b'?\xff\xff\xff\xff\xff\xff'


In [338]:
import time

epoch1970 = const(946684800000)   # micropython uses a yr 2000 epoch

def rhex(h):  
    return (h>>4)*10 + (h&0x0f)
def dhex(v):  
    return chr(((v//10)<<4) + (v%10))

def rtctojsepoch(i2c, busywaitsec=False):
    r = i2c.readfrom_mem(0x68, 0x00, 7)
    year, month, day = rhex(r[6])+2000, rhex(r[5]), rhex(r[4])
    hour, minute, second = rhex(r[2]), rhex(r[1]), rhex(r[0])
    micropythonepoch = time.mktime((year, month, day, hour, minute, second, -1, -1))
    if busywaitsec:   # this is to get the exact transition to next second tick (for calibration timings)
        micropythonepoch += 1000
        while (r[0] == i2c.readfrom_mem(0x68, 0x00, 1)[0]):
            pass
    return micropythonepoch*1000 + epoch1970

def jsepochtortc(jsepoch, i2c):
    micropythonepoch = (jsepoch - epoch1970)//1000
    microsecond = jsepoch % 1000
    year, month, day, hour, minute, second, d1, d2 = time.localtime(micropythonepoch)
    i2c.writeto(0x68, chr(0) + dhex(second) + dhex(minute) + dhex(hour))
    i2c.writeto(0x68, chr(4) + dhex(day) + dhex(month) + dhex(year-2000))
    
def isodatetojsepoch(isodate):  # 2017-11-19T16:05:45.413Z
    #mtime = ure.match("(\d\d\d\d)-(\d\d)-(\d\d)[T ](\d\d)[:c](\d\d)[:c](\d\d)[\.d](\d\d\d)", isodate)
    #year, month, day, hour, minute, second, microsecond = int(mtime.group(1)), int(mtime.group(2)), int(mtime.group(3)), int(mtime.group(4)), int(mtime.group(5)), int(mtime.group(6)), int(mtime.group(7)) 
    year, month, day = int(isodate[:4]), int(isodate[5:7]), int(isodate[8:10])
    hour, minute, second, microsecond = int(isodate[11:13]), int(isodate[14:16]), int(isodate[17:19]), int(isodate[20:23])
    micropythonepoch = time.mktime((year, month, day, hour, minute, second, -1, -1))
    return micropythonepoch*1000 + microsecond + epoch1970

def jsepochtoisodate(jsepoch):
    micropythonepoch = (jsepoch - epoch1970)//1000
    microsecond = jsepoch % 1000
    year, month, day, hour, minute, second, d1, d2 = time.localtime(micropythonepoch)
    return "{0:04d}-{1:02d}-{2:02d}T{3:02d}:{4:02d}:{5:02d}.{6:03d}".format(year, month, day, hour, minute, second, microsecond)
    


In [220]:
jsepoch = isodatetojsepoch("2020-12-12T11:11:11.888")
print(jsepoch)
jsepochtortc(jsepoch, i2c)

1607771471888


In [339]:
print(jsepochtoisodate(rtctojsepoch(i2c)))

2020-12-12T15:58:18.000


In [254]:
print(jsepochtoisodate(rtctojsepoch(i2c)))

2020-12-12T11:20:33.000


In [309]:
%serialconnect

Connecting to --port=/dev/ttyUSB7 --baud=115200 
Ready.


In [310]:
from machine import ADC, Pin
adc = ADC(Pin(35))
adc.atten(ADC.ATTN_11DB)
padc = Pin(21, Pin.OUT)


In [335]:
#padc.value(1)
meas = adc.read()
padc.value(0)
print(meas)


4095


In [363]:
%serialconnect

Connecting to --port=/dev/ttyUSB5 --baud=115200 
Ready.


In [364]:
import machine
fconfig = dict(x.split()  for x in open("config.txt"))
deepsleepsec = int(fconfig["deepsleepsec"])
pinenablewifi = machine.Pin(int(fconfig["pinenablewifi"]), machine.Pin.IN, machine.Pin.PULL_UP)
pinled = machine.Pin(int(fconfig["pinled"]), machine.Pin.OUT)



In [366]:
while 1:
    pinled.value(1-pinenablewifi.value())



*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
KeyboardInterrupt: 


In [362]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.12-87-g96716b46e on 2020-01-24; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f57d98ab950, open=True>(port='/dev/ttyUSB5', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [47]:
%sendtofile --source /home/julian/extrepositories/micropython-async/asyn.py
%sendtofile --source /home/julian/extrepositories/micropython-async/aswitch.py
%sendtofile --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py

Sent 660 lines (23451 bytes) to mqtt_as.py.


In [350]:
%sendtofile config.txt

wifiname        DoESLiverpool
wifipassword    decafbad00
mqttbroker      mqtt.local
boardname       esp32barelake
pinenablemeas   21
pinscl          18
pinsda          17
pinled          2
pinenablewifi   4
pinadcmeas      35
wificonnectsecs 30
deepsleepsec    20

Sent 12 lines (263 bytes) to config.txt.


In [388]:
%serialconnect

Connecting to --port=/dev/ttyUSB5 --baud=115200 
Ready.


In [401]:
%ls

Listing directory '/'.
      139    boot.py
      170    config.txt
             lib/
     4243    main.py
    23452    mqtt_as.py
      960    utils.py


In [400]:
import os
os.rename("main.py", "dd")


In [385]:
%sendtofile main.py

import esp32, machine, time

rtc = machine.RTC()
def ftime():
    d = rtc.datetime()
    return time.mktime((d[0], d[1], d[2], d[4], d[5], d[6], 0, 0)) + (d[7]//1000)/1000

timestartup = ftime()
timeconnected = 0

resetcause = ["0", "PWRON", "HARD", "WDT", "DEEPSLEEP", "SOFT"][machine.reset_cause()]+"_RESET"
wakereason = ["0", "1", "EXT0", "EXT1", "TIMER", "TOUCHPAD", "ULP"][machine.wake_reason()]+"_WAKE"
print("Startup secs=%.3f: %s, %s" % (timestartup, resetcause, wakereason))

fconfig = dict(x.split()  for x in open("config.txt"))
deepsleepsec = int(fconfig["deepsleepsec"])
pinenablewifi = machine.Pin(int(fconfig["pinenablewifi"]), machine.Pin.IN, machine.Pin.PULL_DOWN)
pinled = machine.Pin(int(fconfig["pinled"]), machine.Pin.OUT)

for i in range(100):
    pinled.value((1-pinenablewifi.value())==((i%10)!=5))
    time.sleep_ms(100)

esp32.wake_on_ext0(pinenablewifi, esp32.WAKEUP_ANY_HIGH)
timeatsleep = ftime()

print("time at sleep:", timeatsleep, "deepsleep secs", deepsleepsec)
machine.deepsleep(deepsleepsec*1000)


Sent 28 lines (1033 bytes) to main.py.


In [386]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.11-406-g4a6974bea on 2019-10-06; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f57d8037bd0, open=True>(port='/dev/ttyUSB5', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [82]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Traceback (most recent call last):
  File "main.py", line 1, in <module>
  File "mqtt_as.py", line 30, in <module>
KeyboardInterrupt: 
MicroPython v1.11-406-g4a6974bea on 2019-10-06; ESP32 module with ESP32
Type "help()" for more information.
>>> 
>>> 
raw REPL; CTRL-B to exit
>Ready.


In [92]:
%sendtofile mqttoperation.py

from mqtt_as import MQTTClient, config
import uasyncio as asyncio
import asyn, os, time, machine

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = machine.Pin(int(fconfig["pinled"]), machine.Pin.OUT)
pinled.value(1)
time.sleep(2)
boardname = fconfig["boardname"]
pinled.value(0)

async def publishgentask(topic, gen):
    try:
        for n, l in enumerate(gen):
            await client.publish(topic, l)
        await client.publish(topic+"/done", str(n))
    except Exception as e:
        await client.publish(topic+"/exception", str(e))

def publishgen(subtopic, gen):
    aloop.create_task(publishgentask(boardname+"/"+subtopic, gen))

async def callbackcmdtask(topic, msg):
    print("executing", [msg])
    try:
        exec(msg)
        await client.publish(boardname+'/reply', "1")
    except Exception as e:
        print(e)
        await client.publish(boardname+'/reply/exception', str(e))
    return

def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(topic, msg))

async def mainconnect():
    print("connecting")
    await client.connect()
    print("subscribing")
    await client.subscribe(boardname+'/cmd', 1)
    await client.publish(boardname+"/ip", client._sta_if.ifconfig()[0], retain=True, qos=1)
    while True:
        await client.publish(boardname+'/time', str(time.time()))
        await asyncio.sleep_ms(2000)
    
    
config['subs_cb'] = callbackcmd
config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]

MQTTClient.DEBUG = True
client = MQTTClient(config)

aloop = asyncio.get_event_loop()
aloop.run_until_complete(mainconnect())


Sent 56 lines (1672 bytes) to mqttoperation.py.


In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [94]:
assert False, "Are you sure you want to delete everything?"
try:
    for f in os.ilistdir("data"):
        os.remove("data/"+f[0])
        print(f)
    os.rmdir("data")
except OSError as e:
    print(e)
os.mkdir("data")


Are you sure you want to delete everything?
('d0_0.csv', 32768, 0, 118)
('d1_87.csv', 32768, 0, 132)
('d2_174.csv', 32768, 0, 135)
('d3_260.csv', 32768, 0, 135)
('d4_347.csv', 32768, 0, 135)
('d5_434.csv', 32768, 0, 135)
('d6_521.csv', 32768, 0, 135)


In [95]:
# Low power logging code mode below
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [96]:
%sendtofile main.py

from machine import I2C, Pin, RTC, ADC
import time, machine, ustruct, os

# constants
deepsleepseconds = const(5)
rtkmemorylimit = const(600)
datafile = "data/d%d_%d.csv"
datadir = "data"

# Set up the pins
ledpin = Pin(2, Pin.OUT)
ledpin.value(1)
checklowpin = Pin(21, Pin.IN, Pin.PULL_UP)
if checklowpin.value() == 1:
    import mqttoperation
    raise Exception("Pin21 disconnected, bail out")
    
checklowpin = Pin(21, Pin.IN, Pin.PULL_DOWN)
enablepin = Pin(19, Pin.OUT)
adcpin = Pin(33)
adcval = ADC(adcpin)
adcval.atten(ADC.ATTN_11DB)

# make adc reading
enablepin.value(1)
time.sleep_ms(50)
ledpin.value(0)
time.sleep_ms(50)
measval = adcval.read()
meastime = time.mktime(time.localtime())
enablepin.value(0)

# append to rtkmemory and save file of exceeded
rtc = RTC()
rtcmemstore = bytearray(rtc.memory())
if len(rtcmemstore) == 0:
    t0 = meastime
    rtcmemstore.extend(ustruct.pack("I", meastime))
else:
    t0 = ustruct.unpack("I", rtcmemstore[:4])[0]

print("measurement", meastime, measval, len(rtcmemstore))
    
meastimeoffset = meastime - t0
rtcmemstore.extend(ustruct.pack("HH", (meastimeoffset&0xFFFF), measval))
if len(rtcmemstore) > rtkmemorylimit:
    numfiles = sum(1  for l in os.ilistdir(datadir))
    fname = (datafile % (numfiles, t0))
    print("writing", fname)
    fout = open(fname, "w")
    for i in range(4, len(rtcmemstore), 4):
        mt, mv = ustruct.unpack("HH", rtcmemstore[i:i+4])
        fout.write("%d,%d\n" % (mt+t0, mv))
    fout.close()
    rtcmemstore = b""
rtc.memory(rtcmemstore)
machine.deepsleep(deepsleepseconds*1000)
    

Sent 58 lines (1576 bytes) to main.py.


In [264]:
%sendtofile main.py

from machine import I2C, Pin, RTC, ADC
import time, machine, ustruct
from rtctimer import jsepochtoisodate, rtctojsepoch
import random

time.sleep(2)
deepsleepseconds = 3

rtc = RTC()
recs = bytearray(rtc.memory())
rtcsecs = time.mktime(time.localtime())
print(len(recs), recs[-10:])

adc = ADC(Pin(33))
adc.atten(ADC.ATTN_11DB)
padc = Pin(19, Pin.OUT)
padc.value(1)
meas = adc.read()
padc.value(0)
print(meas, len(recs), recs[-10:])

if len(recs) < 200:
    recs.extend(ustruct.pack("HH", (rtcsecs&0xFFFF), meas))
    rtc.memory(recs)
    machine.deepsleep(deepsleepseconds*1000)


Sent 25 lines (581 bytes) to main.py.


In [255]:
import ustruc
print(random.random())

0.4089468


In [257]:
rtc.memory("")

In [234]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [252]:
print(bytearray(rtc.memory()))

bytearray(b'gggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggg

In [233]:
rtc = RTC()
print(rtc.datetime())
import machine
machine.deepsleep(3000)


(2000, 1, 1, 5, 0, 8, 45, 535935)
ets Jun  8 2016 00:22:57

rst:0x5 (DEEPSLEEP_RESET),boot:0x17 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:2
load:0x3fff0018,len:4
load:0x3fff001c,len:4928
ho 0 tail 12 room 4
load:0x40078000,len:9656
ho 0 tail 12 room 4
load:0x40080400,len:6252
entry 0x400806f4
MicroPython v1.11-406-g4a6974bea on 2019-10-06; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' 

In [228]:
r = i2c.readfrom_mem(0x68, 0x00, 7)
year, month, day = rhex(r[6])+2000, rhex(r[5]), rhex(r[4])
hour, minute, second = rhex(r[2]), rhex(r[1]), rhex(r[0])
micropythonepoch = time.mktime((year, month, day, hour, minute, second, -1, -1))
print(micropythonepoch)

618594005


In [210]:
jsepoch = rtctojsepoch(i2c)
jsepoch

1565278673000


In [143]:
import machine
r = machine.RTC()
help(r)

object <RTC> is of type RTC
  init -- <function>
  datetime -- <function>
  memory -- <function>


In [164]:
import time
help(time)
print(time.localtime())

object <module 'utime'> is of type module
  __name__ -- utime
  localtime -- <function>
  mktime -- <function>
  time -- <function>
  sleep -- <function>
  sleep_ms -- <function>
  sleep_us -- <function>
  ticks_ms -- <function>
  ticks_us -- <function>
  ticks_cpu -- <function>
  ticks_add -- <function>
  ticks_diff -- <function>
(2000, 1, 1, 0, 14, 12, 5, 1)


In [166]:
print(r.datetime())
print(time.localtime())
print(i2c.readfrom_mem(0x68, 0x00, 7))
      

(2000, 1, 1, 5, 0, 14, 24, 767019)
(2000, 1, 1, 0, 14, 24, 5, 1)
b'Y\x12\x00\x01\x01\x01\x00'


In [142]:
def rhex(h):  
    return (h>>4)*10 + (h&0x0f)
r = i2c.readfrom_mem(0x68, 0x00, 7)
year, month, day = rhex(r[6])+2000, rhex(r[5]), rhex(r[4])
hour, minute, second = rhex(r[2]), rhex(r[1]), rhex(r[0])
micropythonepoch = time.mktime((year, month, day, hour, minute, second, -1, -1))
print(micropythonepoch)


b'\x10\x00\x00\x01\x01\x01\x00'


In [41]:
import ustruct, random
b = bytearray()
for i in range(250):
    b.extend(ustruct.pack("If", i, random.uniform(10,15)))
print(b[:100])

import machine
r = machine.RTC()
r.memory(b)
print(len(r.memory()))

bytearray(b'\x00\x00\x00\x00l\x84mA\x01\x00\x00\x00(\xdaQA\x02\x00\x00\x00B\x93mA\x03\x00\x00\x00DyBA\x04\x00\x00\x00\xbe^_A\x05\x00\x00\x00E\xde-A\x06\x00\x00\x00/|`A\x07\x00\x00\x00\xc0\xd6BA\x08\x00\x00\x00\xb96UA\t\x00\x00\x00\xcd\xc2_A\n\x00\x00\x00\xa6\x19@A\x0b\x00\x00\x00\x10\xcdNA\x0c\x00\x00\x00')
2000


In [23]:
import machine
r = machine.RTC()
r.memory(b)

In [24]:
print(r.memory())

b'\x08\x00\x00\x00#oXA\x08\x00\x00\x00j\x80]A\x08\x00\x00\x00\x1aCeA\x08\x00\x00\x008\x05?A\x08\x00\x00\x000;7A\x08\x00\x00\x00J\xe7gA\x08\x00\x00\x00\xd6\xc1]A\x08\x00\x00\x00d3%A\x08\x00\x00\x00\xcaHaA\x08\x00\x00\x00\xf9\xc3\\A\x08\x00\x00\x00\xd6V2A\x08\x00\x00\x00\xeaE*A\x08\x00\x00\x00\xe2\xf9MA\x08\x00\x00\x00\xe0r9A\x08\x00\x00\x00WkSA\x08\x00\x00\x00\xa5\xaa7A\x08\x00\x00\x00\xe2\xbc8A\x08\x00\x00\x00\x85\x8d>A\x08\x00\x00\x00\xd7\xfb/A\x08\x00\x00\x00\x84\xc21A\x08\x00\x00\x00\x9b\xf75A\x08\x00\x00\x00D\xe3QA\x08\x00\x00\x00\xfa\xc18A\x08\x00\x00\x00p:AA\x08\x00\x00\x00\xc4\x0eeA\x08\x00\x00\x00\xc4\xc8+A\x08\x00\x00\x00\xf0\x8bIA\x08\x00\x00\x00gMaA\x08\x00\x00\x00 $EA\x08\x00\x00\x00\xd2\xc5:A\x08\x00\x00\x00\x1ck`A\x08\x00\x00\x00\x06=%A\x08\x00\x00\x00\xff\xfcjA\x08\x00\x00\x00\xf8/4A\x08\x00\x00\x00\x96J:A\x08\x00\x00\x00\x8c\xdb_A\x08\x00\x00\x00\x14a9A\x08\x00\x00\x00\xe9\xca+A\x08\x00\x00\x00\x82\xaboA\x08\x00\x00\x00\xda\xd6[A\x08\x00\x00\x00s\xce%A\x08\x00\x00\x00\n

In [25]:
print(len(r.memory()))

960


In [30]:
print(24*20)

480


In [28]:
import machine
r = machine.RTC()
print(r.memory())

b'\x08\x00\x00\x00#oXA\x08\x00\x00\x00j\x80]A\x08\x00\x00\x00\x1aCeA\x08\x00\x00\x008\x05?A\x08\x00\x00\x000;7A\x08\x00\x00\x00J\xe7gA\x08\x00\x00\x00\xd6\xc1]A\x08\x00\x00\x00d3%A\x08\x00\x00\x00\xcaHaA\x08\x00\x00\x00\xf9\xc3\\A\x08\x00\x00\x00\xd6V2A\x08\x00\x00\x00\xeaE*A\x08\x00\x00\x00\xe2\xf9MA\x08\x00\x00\x00\xe0r9A\x08\x00\x00\x00WkSA\x08\x00\x00\x00\xa5\xaa7A\x08\x00\x00\x00\xe2\xbc8A\x08\x00\x00\x00\x85\x8d>A\x08\x00\x00\x00\xd7\xfb/A\x08\x00\x00\x00\x84\xc21A\x08\x00\x00\x00\x9b\xf75A\x08\x00\x00\x00D\xe3QA\x08\x00\x00\x00\xfa\xc18A\x08\x00\x00\x00p:AA\x08\x00\x00\x00\xc4\x0eeA\x08\x00\x00\x00\xc4\xc8+A\x08\x00\x00\x00\xf0\x8bIA\x08\x00\x00\x00gMaA\x08\x00\x00\x00 $EA\x08\x00\x00\x00\xd2\xc5:A\x08\x00\x00\x00\x1ck`A\x08\x00\x00\x00\x06=%A\x08\x00\x00\x00\xff\xfcjA\x08\x00\x00\x00\xf8/4A\x08\x00\x00\x00\x96J:A\x08\x00\x00\x00\x8c\xdb_A\x08\x00\x00\x00\x14a9A\x08\x00\x00\x00\xe9\xca+A\x08\x00\x00\x00\x82\xaboA\x08\x00\x00\x00\xda\xd6[A\x08\x00\x00\x00s\xce%A\x08\x00\x00\x00\n

In [191]:
print(jsepochtoisodate(rtctojsepoch(i2c)))

2019-08-08T15:33:08.000


In [182]:
print(jsepochtortc(isodatetojsepoch("2019-08-08T15:32:22.000"), i2c))

None


This is the for connecting to a pressure sensor 
at the bottom of a lake and logging it to its own 
memory.

For now it will do it to MQTT

In [124]:
%serialconnect

Connecting to --port=/dev/ttyUSB7 --baud=115200 
Ready.


In [125]:
%sendtofile main.py

import machine, time, network, os

sleepperiod = const(1)
mqtt_local = "10.0.29.167"


#
# Get the RTC working
#
i2c = machine.I2C(scl=machine.Pin(4), sda=machine.Pin(5)) 
epoch1970 = const(946684800000)   # esp8266 has 2000 epoch
def rhex(h):  return (h>>4)*10 + (h&0x0f)
def rtctojsepoch():
    r = i2c.readfrom_mem(0x68, 0x00, 7)
    year, month, day = rhex(r[6])+2000, rhex(r[5]), rhex(r[4])
    hour, minute, second = rhex(r[2]), rhex(r[1]), rhex(r[0])
    microsecond = 0
    micropythonepoch = time.mktime((year, month, day, hour, minute, second, -1, -1))
    return micropythonepoch*1000 + microsecond + epoch1970   # esp8266 has 2000 

def jsepochtoisodate(jsepoch):
    micropythonepoch = (jsepoch - epoch1970)//1000
    microsecond = jsepoch % 1000
    year, month, day, hour, minute, second, d1, d2 = time.localtime(micropythonepoch)
    return "{0:04d}-{1:02d}-{2:02d}T{3:02d}:{4:02d}:{5:02d}.{6:03d}".format(year, month, day, hour, minute, second, microsecond)

#
# make name of datafile
#
for i in range(100):
    fname = "data%03d.txt" % i
    try:     os.stat(fname)
    except OSError:  break
print(fname)

#
# connect MQTT
#
sta_if = network.WLAN(network.STA_IF)
sta_if.active(True)
sta_if.connect("DoESLiverpool", "decafbad00")
while not sta_if.isconnected():
    pass

print("connected to wifi")
from umqtt.robust import MQTTClient
client = MQTTClient("lakelogger", mqtt_local)
client.connect()
print("connected to mqtt")


#
# Connect to ADC
#
adc = machine.ADC(0)


#
# Main working loop
#
n = 0
fout = None
while True:
    n += 1
    aval = adc.read()
    if fout is None:
        fout = open(fname, "aw")
    client.publish(b"lakelogger/level", str(aval))
    try:
        ts = jsepochtoisodate(rtctojsepoch())
    except OSError as e:
        ts = "unknown"
    fout.write("%s %d\n" % (ts, aval))
    if (n%10) == 0:
        fout.flush()
        fout.close()
        fout = None

    time.sleep(sleepperiod)
    

Sent 80 lines (1938 bytes) to main.py.


serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB7 --baud=115200 
Ready.


In [127]:
# Check what files we have
%serialconnect
import os
for f in os.ilistdir():
    if f[0][0] == "d":
        print(f[0], f[3])


Connecting to --port=/dev/ttyUSB7 --baud=115200 
Ready.
data000.txt 0


In [126]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.4-8-ga9a3caad0 on 2018-05-11; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f72a1c13390, open=True>(port='/dev/ttyUSB7', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [103]:
%fetchfile --binary data000.txt

Fetched 22680=22680 bytes from data000.txt.
Saving file to 'data000.txt'

In [238]:
%serialconnect

Connecting to --port=/dev/ttyUSB6 --baud=115200 
Ready.


In [211]:
import machine
print(machine.freq())

80000000


In [239]:
%sendtofile main.py

#
# Demo of deep sleep cycle (pin16 to RST on ESP8266)
#
import machine
st = str(machine.RTC().datetime())
f = open("log.txt", "a")
f.write(st)
print(st)
if machine.reset_cause() == machine.DEEPSLEEP_RESET:
    f.write('  woke from a deep sleep\n')
else:
    f.write('  power on or hard reset\n')
f.close()

import time
p = machine.Pin(4, machine.Pin.OUT)
for i in range(20):
    p.value(i%2)
    time.sleep(0.2)
p.init(machine.Pin.IN)

# configure RTC.ALARM0 to be able to wake the device
machine.deepsleep(sleep_ms=5000)


Sent 23 lines (523 bytes) to main.py.


In [233]:
%serialconnect

Connecting to --port=/dev/ttyUSB6 --baud=115200 
Ready.


In [229]:
#%sendtofile main.py

import time, machine
#p = machine.Pin(2, machine.Pin.OUT)
p = machine.Pin(4, machine.Pin.OUT)
for i in range(221):
    p.value(i%2)
    time.sleep(0.6)




*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 8, in <module>
KeyboardInterrupt: 


In [230]:
p.init(machine.Pin.IN)

In [177]:
for l in open("log.txt"):
    print(l, end="")

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
OSError: [Errno 2] ENOENT


In [161]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.4-8-ga9a3caad0 on 2018-05-11; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f72a1c133c8, open=True>(port='/dev/ttyUSB7', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [ ]:
#
# Call this to set the time on the external RTC
#
import ure
def dhex(v):  return chr(((v//10)<<4) + (v%10))
def jsepochtortc(jsepoch):
    micropythonepoch = (jsepoch - epoch1970)//1000
    microsecond = jsepoch % 1000
    year, month, day, hour, minute, second, d1, d2 = time.localtime(micropythonepoch)
    i2c.writeto(0x68, chr(0) + dhex(second) + dhex(minute) + dhex(hour))
    i2c.writeto(0x68, chr(4) + dhex(day) + dhex(month) + dhex(year-2000))
def isodatetojsepoch(isodate):
    mtime = ure.match("(\d\d\d\d)-(\d\d)-(\d\d)[T ](\d\d)[:c](\d\d)[:c](\d\d)[\.d](\d\d\d)", isodate)
    year, month, day, hour, minute, second, microsecond = int(mtime.group(1)), int(mtime.group(2)), int(mtime.group(3)), int(mtime.group(4)), int(mtime.group(5)), int(mtime.group(6)), int(mtime.group(7)) 
    micropythonepoch = time.mktime((year, month, day, hour, minute, second, -1, -1))
    return micropythonepoch*1000 + microsecond + epoch1970
#jsepochtortc(isodatetojsepoch("2018-11-16T13:00:00.000"))
